In [1]:
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql import SparkSession

default_spark_config = {
    'master':'yarn',
    'spark.app.name': 'Bigquery Token Price' , #getNotebookName()
    "spark.yarn.maxAppAttempts":"1",
    "spark.task.cpus":"2",
    "spark.driver.cores":"1",
    "spark.driver.memory":"4g",
    "spark.executor.memory":"4g",
    "spark.executor.pyspark.memory":"4g",
    "spark.executor.cores":"8",
    "spark.cores.max":"8",
    "spark.sql.repl.eagerEval.enabled":True,
    "spark.sql.execution.arrow.pyspark.enabled":True,
    "spark.jars.packages": "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.25.1",
    'spark.jars': 'gs://fp_spark_cluster/jars/spark-doris-connector-3.1_2.12-1.0.0-SNAPSHOT.jar'
}

def getSpark(spark_config = default_spark_config):
    builder = SparkSession.builder
    for k in spark_config.keys():
        builder = builder.config(k,spark_config[k])
    spark = builder.enableHiveSupport()\
     .getOrCreate()
    return spark

spark = getSpark()


:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-36c3986e-df54-4117-80ba-e2a6fd5e977e;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.25.1 in central
:: resolution report :: resolve 199ms :: artifacts dl 3ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.25.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrie

In [24]:
def read_data(table: str, partition_filter: str=None ):
    df_origin = spark.read.format("bigquery")\
        .option("viewsEnabled", True)\
        .option("filter", partition_filter)\
        .load(table) 
    
    df = df_origin
    return df

In [29]:
df = read_data(table= "gaia-data.struct_data.fantom_top_dex_swap_view", partition_filter=f"""block_time =  '2022-04-10T01:15:50'  """)



In [30]:
df.show(1)

+-------------------+--------------+--------------+--------------+--------------+--------------------+--------------------+------------+
|         block_time|token_a_symbol|token_b_symbol|token_a_amount|token_b_amount|     token_a_address|     token_b_address|block_number|
+-------------------+--------------+--------------+--------------+--------------+--------------------+--------------------+------------+
|2022-04-10T01:15:50|          USDC|          WFTM|      0.388702|           0.3|0x04068da6c83afcf...|0x21be370d5312f44...|    35675751|
+-------------------+--------------+--------------+--------------+--------------+--------------------+--------------------+------------+
only showing top 1 row



In [31]:
df2 = read_data(table= "footprint-test-341610.airflow_dbt_test.fantom_top_dex_swap_view", partition_filter=f"""block_time =  '2022-04-10T01:15:50'  """ )
df2.show(1)

Py4JJavaError: An error occurred while calling o181.showString.
: com.google.cloud.bigquery.connector.common.BigQueryConnectorException: Error creating destination table using the following query: [SELECT `block_time`,`token_a_symbol`,`token_b_symbol`,`token_a_amount`,`token_b_amount`,`token_a_address`,`token_b_address`,`block_number` FROM `footprint-test-341610.airflow_dbt_test.fantom_top_dex_swap_view` WHERE ((block_time =  '2022-04-10T01:15:50'  ))]
	at com.google.cloud.bigquery.connector.common.BigQueryClient.materializeTable(BigQueryClient.java:451)
	at com.google.cloud.bigquery.connector.common.BigQueryClient.materializeViewToTable(BigQueryClient.java:434)
	at com.google.cloud.bigquery.connector.common.ReadSessionCreator.getActualTable(ReadSessionCreator.java:150)
	at com.google.cloud.bigquery.connector.common.ReadSessionCreator.getActualTable(ReadSessionCreator.java:136)
	at com.google.cloud.bigquery.connector.common.ReadSessionCreator.create(ReadSessionCreator.java:69)
	at com.

In [32]:
def read_data2(table: str, partition_filter: str=None ):
    df_origin = spark.read.format("bigquery")\
        .option("viewsEnabled", True)\
        .load(table) 
    
    df = df_origin
    return df

In [33]:
df2 = read_data2(table= "footprint-test-341610.crypto_bsc.uniswap_cashflow_swap_test")
df2.show(1)

Py4JJavaError: An error occurred while calling o189.showString.
: com.google.cloud.bigquery.connector.common.BigQueryConnectorException: Error creating destination table using the following query: [SELECT `test` FROM `footprint-test-341610.crypto_bsc.uniswap_cashflow_swap_test` ]
	at com.google.cloud.bigquery.connector.common.BigQueryClient.materializeTable(BigQueryClient.java:451)
	at com.google.cloud.bigquery.connector.common.BigQueryClient.materializeViewToTable(BigQueryClient.java:434)
	at com.google.cloud.bigquery.connector.common.ReadSessionCreator.getActualTable(ReadSessionCreator.java:150)
	at com.google.cloud.bigquery.connector.common.ReadSessionCreator.getActualTable(ReadSessionCreator.java:136)
	at com.google.cloud.bigquery.connector.common.ReadSessionCreator.create(ReadSessionCreator.java:69)
	at com.google.cloud.spark.bigquery.direct.BigQueryRDDFactory.createRddFromTable(BigQueryRDDFactory.java:126)
	at com.google.cloud.spark.bigquery.direct.DirectBigQueryRelation.buildSca